In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import MicroTools as micro
from MicroTools import plot_tools as pt
import MiniTools as mini
import param_scan

from importlib.resources import open_text


In [7]:
# settings for Fig. 1
TOTAL_RATE = False
INCLUDE_MB_LAST_BIN = False
STACKED = False
PLOT_FAMILY=False
colormap=cm.Blues
TEMP1_COLOR = 'deeppink'
TEMP2_COLOR = 'dodgerblue'

PATH_PLOTS = 'plots/event_rates/'

In [47]:
params = {'g': 1, 'Ue4Sq': 0.1, 'Um4Sq': 3e-2/4, 'm4': 1}

rates_dic_osc = param_scan.get_nue_rates(params, oscillations=True, use_numu_MC=False, include_antineutrinos=True, decay=False, decouple_decay=True, disappearance=False, energy_degradation=False, n_replications=1)
rates_dic_decay = param_scan.get_nue_rates(params, oscillations=True, use_numu_MC=False, include_antineutrinos=True, decay=True, decouple_decay=True, disappearance=False, energy_degradation=False, n_replications=1)

In [1]:


fig, ax1 = pt.std_fig(figsize=(3.2*1.2,2.5*1.2))
bins = param_scan.MB_Ereco_official_bins
bin_w = np.diff(bins)
bin_c = bins[:-1] + bin_w/2

######################################
# MiniBooNE 
if TOTAL_RATE:
    units= 1
    ax1.set_ylabel(r"Events")
else:
    units = 1/micro.bin_width
    ax1.set_ylabel(r"Events/MeV")

nue_data = np.genfromtxt(
    open_text(
        f"MiniTools.include.MB_data_release_2020.combined",
        f"miniboone_nuedata_lowe.txt",
    )
)
nue_bkg = np.genfromtxt(
            open_text(
                f"MiniTools.include.MB_data_release_2020.combined",
                f"miniboone_nuebgr_lowe.txt",
            )
        )

# plot data
pt.data_plot(ax1, 
          X=bin_c,
          Y=nue_data*units, 
          xerr=bin_w/2, 
          yerr=np.sqrt(nue_data)*units,
          zorder=3)


ax1.hist(bins[:-1],
        bins=bins,
        weights=(nue_bkg)*units,
        edgecolor='black',
        color='dodgerblue',
        lw=0.5,
        label=r'Backgrounds',
        histtype='stepfilled',
        zorder=2)


# plot sterile
ax1.hist(bins[:-1],
        bins=bins,
        weights=(rates_dic_osc['MC_nue_app'] + nue_bkg)*units,
        edgecolor='orange',
        color='white',
        lw=0.5,
        label=r'3+1 best-fit',
        histtype='stepfilled',
        zorder=1.5)

ax1.hist(bins[:-1],
        bins=bins,
        weights=(rates_dic_decay['MC_nue_app'] + nue_bkg)*units,
        edgecolor='deeppink',
        color='white',
        lw=0.5,
        label=r'3+1 best-fit',
        histtype='stepfilled',
        zorder=1)



ax1.legend(fontsize=8, markerfirst=False, ncol=1)
ax1.annotate(text=r'MiniBooNE 2020', xy=(0.0,1.05), xycoords='axes fraction', fontsize=9)
ax1.set_xlabel(r"Reconstructed $E_\nu^{\rm QE}$ (GeV)", fontsize=9, labelpad=2.5)
if INCLUDE_MB_LAST_BIN:
    ax1.set_xticks([0.2,0.5,1,1.5,2,2.5,3])
    ax1.set_xlim(0.2,3)
else:
    ax1.set_xticks([0.2,0.4,0.6,0.8,1,1.2,1.4])
    ax1.set_xlim(0.2,1.5)

# ax1.xaxis.set_major_locator(pt.MultipleLocator(0.5))
# ax1.xaxis.set_minor_locator(pt.MultipleLocator(0.1))

ax1.annotate(text=r'Kelly\,et.\,al.\,2024', xy=(0.65,1.05), xycoords='axes fraction', fontsize=8.5)

fig.savefig(f"{PATH_PLOTS}/Fig_1.png", dpi=400)
fig.savefig(f"{PATH_PLOTS}/Fig_1.pdf")

NameError: name 'pt' is not defined